## Comparison of clearsky models

In [1]:
import pandas as pd
import numpy as np
import feather
import pvlib
import sys
sys.path.append('..')
from src.utils.download_data import timer

In [2]:
DATA_PATH = '/home/SHARED/SOLAR/data/'

In [3]:
# read minute data and location info
df = pd.read_pickle(DATA_PATH + 'oahu_min_cs.pkl')

In [4]:
df.head()

,Radiation,GHI,GTI,Pysolar,Altitude,Ineichen,Haurwitz,Solis
Location,Datetime,,,,,,,
AP1,2010-03-19 14:15:00-10:00,476.328,NaN,973.629240,58.317632,840.960548,871.749955,907.496424
AP3,2010-03-19 14:15:00-10:00,382.777,NaN,973.625962,58.315987,841.600950,871.733426,907.477590
AP4,2010-03-19 14:15:00-10:00,351.610,NaN,973.624679,58.315343,841.594000,871.726958,907.470220
AP5,2010-03-19 14:15:00-10:00,390.092,NaN,973.630994,58.318513,841.628209,871.758798,907.506499
AP6,2010-03-19 14:15:00-10:00,353.928,343.313,973.628714,58.317368,841.615852,871.747297,907.493395


In [5]:
df.shape

(9058229, 7)

Replace negative values with 0

In [6]:
df['GHI'] = df['GHI'].where(df['GHI'] >= 0, other=0)

In [7]:
df1 = df.unstack(level='Location').between_time('7:30', '17:30').stack(level='Location')
df1.shape

(6041579, 7)

Compute Haurwitz and Kasten clearsky based on sun altitude:

    ghi_csm = 910 * math.sin(math.radians(altitude))`
    ghi_csm = 1098 * math.sin(math.radians(altitude)) * math.exp(-0.057 / (math.sin(math.radians(altitude))))`

In [8]:
df1['Kasten_UCM']   = 910 * np.sin(np.radians(df1['Altitude']))
df1['Haurwitz_UCM'] = 1098 * np.sin(np.radians(df1['Altitude'])) * np.exp(-0.057 / (np.sin(np.radians(df1['Altitude']))))

In [9]:
df1.describe().style

Radiation,GHI,GTI,Pysolar,Altitude,Ineichen,Haurwitz,Solis,Kasten_UCM,Haurwitz_UCM
count,6.04158e+06,710774,6.04158e+06,6.04158e+06,6.04158e+06,6.04158e+06,6.04158e+06,6.04158e+06,6.04158e+06
mean,533.703,436.457,853.326,46.8803,632.429,693.075,698.903,625.453,695.06
std,318.703,345.336,116.846,19.6568,244.777,244.64,258.32,204.019,244.74
min,0,0.363829,96.9197,3.1376,8.27051,20.5473,25.4578,49.808,21.2122
25%,262.88,138.742,802.607,31.2873,442.119,508.995,497.834,472.59,510.95
50%,469.039,315.365,878.992,45.8307,677.998,725.391,739.376,652.728,727.414
75%,790.074,734.666,931.047,61.7907,844.882,904.928,924.851,801.916,906.982
max,1700.35,1587.02,1022.68,89.8983,976.033,1035.09,1061.95,909.999,1037.16


Statistics for the different clearsky models:
  * **Pysolar**: model from `pysolar` library
  * **Ineichen**: `pvlib`'s Ineichen model
  * **Haurwitz**: `pvlib`'s Haurwitz model
  * **Solis**: `pvlibs`'s simplified Solis model
  * **Haurwitz_UCM**: Haurwitz model computed using formula taken from UCM code
  * **Kasten_UCM**: Kasten model computed using formula taken from UCM code

We compute the main descriptive statistics plus the number of times the GHI is greater than the clearsky model. We also check if the clearsky model returns a GHI of 0 when the sensor has a value greater than 0 (`#CS=0`)

In [10]:
res = []
for clearsky in ('Pysolar', 'Ineichen', 'Haurwitz', 'Solis', 'Haurwitz_UCM', 'Kasten_UCM'):   
    name = 'GHI_{}'.format(clearsky)
    cs = df1.loc[~np.isclose(df1['GHI'], 0) &  np.isclose(df1[clearsky], 0), ['GHI', clearsky]]
    
    df1[name] = np.where(np.isclose(df1[clearsky], 0), 1, df1['GHI']/df1[clearsky])
    
    summ = df1[name].describe()
    q90, q99 = df1[name].quantile(q=[0.9, 0.99])
    per_gt_1, num_gt_1 = df1[name].gt(1).agg([np.mean, np.sum])
    
    summ['90%'] = q90
    summ['99%'] = q99
    summ['#>1'] = num_gt_1
    summ['%>1'] = per_gt_1
    summ['#CS=0'] = cs.shape[0]
    summ.name = clearsky
    res.append(summ)

pd.concat(res, axis=1, sort=False).style

,Pysolar,Ineichen,Haurwitz,Solis,Haurwitz_UCM,Kasten_UCM
count,6.04158e+06,6.04158e+06,6.04158e+06,6.04158e+06,6.04158e+06,6.04158e+06
mean,0.609836,0.872554,0.771898,0.772333,0.769217,0.841759
std,0.339151,0.403347,0.34579,0.345167,0.344605,0.381679
min,0,0,0,0,0,0
25%,0.318779,0.478783,0.425978,0.426563,0.4245,0.463732
50%,0.561595,1.05537,0.933943,0.95859,0.929595,0.968941
75%,0.868854,1.17201,1.04484,1.03738,1.0416,1.15798
max,3.16413,29.1443,12.4256,12.4105,12.1471,7.92473
90%,1.10763,1.28658,1.14367,1.1355,1.14023,1.27142
99%,1.36761,1.63198,1.34406,1.34148,1.33956,1.48835


For the `detect_clearsky()` function we need to do it per station and day (it does not support unequal differences in index)

In [11]:
with timer():
    cs = pvlib.clearsky.detect_clearsky(df1.loc[('2010-03-20', 'AP1'), 'GHI'], 
                                        df1.loc[('2010-03-20', 'AP1'), 'Pysolar'], 
                                        df1.loc[('2010-03-20', 'AP1'), 'GHI'].index.get_level_values('Datetime'), 
                                        10)

/home/atorres/.local/lib/python3.6/site-packages/numpy/lib/function_base.py:1238: FutureWarning: Converting timezone-aware DatetimeArray to timezone-naive ndarray with 'datetime64[ns]' dtype. In the future, this will return an ndarray with 'object' dtype where each element is a 'pandas.Timestamp' with the correct 'tz'.
	To accept the future behavior, pass 'dtype=object'.
	To keep the old behavior, pass 'dtype="datetime64[ns]"'.
  a = asanyarray(a)


Elapsed time (s): 30.057247


In [12]:
df1.loc[('2010-03-20', 'AP1'), ['GHI', 'Pysolar']].loc[cs.to_numpy()].head()

,Radiation,GHI,Pysolar
Datetime,Location,,
2010-03-20 14:09:00-10:00,AP1,936.701,974.431505
2010-03-20 14:10:00-10:00,AP1,934.499,974.101735
2010-03-20 14:11:00-10:00,AP1,931.564,973.767387
2010-03-20 14:12:00-10:00,AP1,933.399,973.428427
2010-03-20 14:13:00-10:00,AP1,934.866,973.084823


For this day and station, the function detects that the model is overestimating the irradiance and thus the clearsky model should probably be set to the maximum?

Finally, we go back to wide form, with one column per sensor, using the GHI normalized by the pysolar clearsky model:

In [16]:
(df1.pivot_table(index='Datetime', columns='Location', values='GHI_Pysolar')
    .drop(columns='AP3')
    .to_pickle(DATA_PATH + 'oahu_min_final.pkl'))